#	Library

In [1]:
import os
import sys
import re
import time
import gc
import glob
import json
from pathlib import Path
from datetime import datetime, timezone, timedelta
from itertools import repeat
import subprocess
import multiprocessing
import warnings
warnings.filterwarnings(action='ignore')
#------------------------------------------------------------
# Third-party packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.core.interactiveshell import InteractiveShell
from ccdproc import ImageFileCollection
import psutil
#	Astropy
from astropy.io import fits
import astropy.io.ascii as ascii
from astropy import units as u
from astropy.table import Table, vstack, hstack
from astropy.coordinates import SkyCoord
from astropy.time import Time

In [2]:
#------------------------------------------------------------
#	plot setting
#------------------------------------------------------------
plt.ioff()
InteractiveShell.ast_node_interactivity = "last_expr"
#------------------------------------------------------------
mpl.rcParams["axes.titlesize"] = 14
mpl.rcParams["axes.labelsize"] = 20
plt.rcParams['savefig.dpi'] = 500
plt.rc('font', family='serif')
#------------------------------------------------------------
os.environ['TZ'] = 'Asia/Seoul'
time.tzset()
start_localtime = time.strftime('%Y-%m-%d_%H:%M:%S_(%Z)', time.localtime())

#	Generate Mask Images

In [3]:
def create_mask_images(input_image, mask_suffix="mask.fits", force_run=False):
	mask_filename = input_image.replace("fits", mask_suffix)
	if (not os.path.exists(mask_filename)) | (force_run == True):
		data = fits.getdata(input_image)
		mask = np.zeros_like(data, dtype=int)
		mask[data == 0] = 1
		mask[data != 0] = 0
		fits.writeto(mask_filename, mask.astype(np.int8), overwrite=True)
	return mask_filename

def combine_or_mask(in_mask_image, ref_mask_image, mask_suffix="all_mask.fits"):
	inmask = fits.getdata(in_mask_image)
	refmask = fits.getdata(ref_mask_image)
	mask = np.logical_or.reduce([inmask, refmask])
	mask_filename = in_mask_image.replace("mask.fits", mask_suffix)
	fits.writeto(mask_filename, mask.astype(np.int8), overwrite=True)
	return mask_filename

# Path

In [4]:
n_binning = 1
path_base = '/lyman/data1/factory'
path_root = '/home/gp/gppy'
path_src = '/home/gp/gppy/src'

path_processed = f'{path_base}/../processed_{n_binning}x{n_binning}_gain2750'
	
path_ref = f'{path_base}/ref_frame'
path_factory = f'{path_base}/tmp4subtraction'

path_config = f"{path_root}/config"
path_keys = path_config  
path_default_gphot = f'{path_config}/gphot.7dt01_{n_binning}x{n_binning}.config'

path_phot_sub = f'{path_src}/phot/gregorydet_7DT_NxN.py'
path_subtraction = f"{path_src}/util/gregorysubt_7DT.py"

# Data

In [9]:
tilenames = [
	"T00138",
	"T00139",
	"T00174",
	"T00175",
	"T00176",
	"T00215",
	"T00216",
]
print(f"{len(tilenames)} tiles set")

7 tiles set


In [12]:
stacked_images = []

for tile in tilenames:
	images = sorted(glob.glob(f"{path_processed}/{tile}/7DT??/*/calib*com.fits"))
	print(f"[{tile}] {len(images)} stacked images found")
	stacked_images += images

print(f"Total {len(stacked_images)} found")

[T00138] 762 stacked images found
[T00139] 761 stacked images found
[T00174] 768 stacked images found
[T00175] 739 stacked images found
[T00176] 759 stacked images found
[T00215] 775 stacked images found
[T00216] 761 stacked images found
Total 5325 found


#	Ready

In [ ]:
reference_images = []
sci_mask_images = []
ref_mask_images = []
all_mask_images = []

for stack_image in stacked_images:
	part = os.path.basename(stack_image).split("_")
	obj = part[2]
	filte = part[5]
	path_ref_frame = f"{path_ref}/{filte}"
	# _reference_images = []
	# for ref_src in ['7DT', 'PS1']: ref_PS1_T14548_00000000_000000_r_0.fits
	_reference_images_ps1 = glob.glob(f"{path_ref_frame}/ref_PS1_{obj}_*_*_{filte}_0.fits")
	_reference_images_7dt = glob.glob(f"{path_ref_frame}/ref_7DT_{obj}_*_*_{filte}_*.fits")
	_reference_images = _reference_images_7dt + _reference_images_ps1

	if len(_reference_images) > 0:
		ref_image = _reference_images[0]
		#	Run
		sci_mask_image = create_mask_images(stack_image)
		ref_mask_image = create_mask_images(ref_image)
		all_mask_image = combine_or_mask(sci_mask_image, ref_mask_image, mask_suffix="all_mask.fits")
	else:
		ref_image = None
		sci_mask_image = None
		ref_mask_image = None
		all_mask_image = None
	#	Save
	reference_images.append(ref_image)
	sci_mask_images.append(sci_mask_image)
	ref_mask_images.append(ref_mask_image)
	all_mask_images.append(all_mask_image)


In [ ]:
#======================================================================
#	Image Subtraction
#======================================================================
for ss, (inim, refim, inmask_image, refmask_image, allmask_image) in enumerate(zip(stacked_images, reference_images, sci_mask_images, ref_mask_images, all_mask_images)):
	if refim != None:
		#	Subtraction Command
		subtraction_com = f"python {path_subtraction} {inim} {refim} {inmask_image} {refmask_image} {allmask_image}"
		print(subtraction_com)
		os.system(subtraction_com)
		#	Outputs
		hdim = inim.replace("fits", "subt.fits")

		_hcim = f"{os.path.basename(refim).replace('fits', 'conv.fits')}"
		dateobs = os.path.basename(inim).split("_")[3]
		timeobs = os.path.basename(inim).split("_")[4]
		part_hcim = _hcim.split("_")
		part_hcim[3] = dateobs
		part_hcim[4] = timeobs
		hcim = f"{path_data}/{'_'.join(part_hcim)}"

		#	Photometry Command for Subtracted Image
		phot_subt_com = f"python {path_phot_sub} {hdim} {inmask_image}"
		print(phot_subt_com)
		os.system(phot_subt_com)
		#	Transient Search Command --> Skip
		# search_com = f"python {path_find} {inim} {refim} {hcim} {hdim}"
		# print(search_com)
		# os.system(search_com)



In [ ]:
#======================================================================
#	Clean the data
#======================================================================
file_to_remove_pattern_list = [
	#	Reduced Images
	'fdz*',
	#	Reduced Images & WCS Files
	'afdz*',
	#	Zero & Dark Corrected Flat Files
	# 'dz*',
	#	Inverted images
	'*inv*',
	#	Tmp catalogs
	'*.pre.cat',
	'*.image.list',
	#	backup header
	'*.head.bkg',


]

for file_to_remove_pattern in file_to_remove_pattern_list:
	rmcom = f'rm {path_data}/{file_to_remove_pattern}'
	print(rmcom)
	os.system(rmcom)
# %%
#======================================================================
#	File Transfer
#======================================================================
t0_data_transfer = time.time()

from concurrent.futures import ThreadPoolExecutor
import shutil
from functools import partial

# def move_file(file_path, destination_folder):
#     file_name = os.path.basename(file_path)
#     shutil.move(file_path, f"{destination_folder}/{file_name}")
def move_file(file_path, destination_folder):
	file_name = os.path.basename(file_path)
	destination_path = os.path.join(destination_folder, file_name)
	shutil.move(file_path, destination_path)
	print(f"Moved {file_path} to {destination_path}")
#----------------------------------------------------------------------
ic_all = ImageFileCollection(path_data, glob_include='calib*.fits', keywords=['object', 'filter',])
#----------------------------------------------------------------------
#	Header File
#----------------------------------------------------------------------
image_files = [f"{path_data}/{inim}" for inim in ic_all.summary['file']]
for ii, inim in enumerate(image_files):
	header_file = inim.replace('fits', 'head')
	imheadcom = f"imhead {inim} > {header_file}"
	subprocess.run(imheadcom, shell=True)
	print(f"[{ii:>4}/{len(image_files):>4}] {inim} --> {header_file}", end='\r')
print()
#----------------------------------------------------------------------
objarr = np.unique(ic_all.summary['object'][~ic_all.summary['object'].mask])
print(f"OBJECT Numbers: {len(objarr)}")

#	Transient Candidates
for oo, obj in enumerate(objarr):
	print("-"*60)
	print(f"[{oo:>4}/{len(objarr):>4}] OBJECT: {obj}")
	_filterarr = list(np.unique(ic_all.filter(object=obj).summary['filter']))
	print(f"FILTER: {_filterarr} ({len(_filterarr)})")
	#	Filter
	for filte in _filterarr:
		#	Path to Destination
		path_destination = f'{path_processed}/{obj}/{obs}/{filte}'
		#
		path_phot = f"{path_destination}/phot"
		path_transient = f"{path_destination}/transient"
		path_transient_cand_png = f"{path_transient}/png_image"
		path_transient_cand_fits = f"{path_transient}/fits_image"
		path_header = f"{path_destination}/header"

		#	Check save path
		paths = [path_destination, path_phot, path_transient, path_transient_cand_png, path_transient_cand_fits, path_header]
		for path in paths:
			if not os.path.exists(path):
				os.makedirs(path)
		#------------------------------------------------------------
		#	Files
		#------------------------------------------------------------
		#	Single Frames
		#------------------------------------------------------------
		#	calib_7DT01_T09373_20240423_032804_r_120.fits
		# single_frames = sorted(glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.fits"))
		single_frames = sorted([f for f in glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.fits") if not re.search(r'\.com\.', f)])
		print(f"Single Frames: {len(single_frames)}")
		# single_phot_catalogs = sorted(glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.phot.cat"))
		single_phot_catalogs = sorted([f for f in glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.phot.cat") if not re.search(r'\.com\.', f)])
		print(f"Phot Catalogs (single): {len(single_phot_catalogs)}")
		single_phot_pngs = sorted([f for f in glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*MAG_*.png") if not re.search(r'\.com\.', f)])
		print(f"Phot PNGs (single): {len(single_phot_pngs)}")
		#------------------------------------------------------------
		#	Stacked Frames
		#------------------------------------------------------------
		stack_frames = sorted(glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.com.fits"))
		print(f"Stack Frames: {len(stack_frames)}")
		stack_phot_catalogs = sorted(glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.com.phot.cat"))
		print(f"Phot Catalogs (stack): {len(stack_phot_catalogs)}")
		stack_phot_pngs = sorted(glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.com.MAG_*.png"))
		print(f"Phot PNGs (stack): {len(stack_phot_pngs)}")
		#------------------------------------------------------------
		#	Subtracted Frames
		#------------------------------------------------------------
		#	ref_PS1_T09373_00000000_000000_r_0.conv.fits
		convolved_ref_frames = sorted(glob.glob(f"{path_data}/ref_*_{obj}_*_*_{filte}_*.conv.fits"))
		print(f"Convolved Ref Frames: {len(convolved_ref_frames)}")
		subt_frames = sorted(glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.subt.fits"))
		print(f"Subt Frames: {len(subt_frames)}")
		subt_phot_catalogs = sorted(glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.subt.phot.cat"))
		print(f"Phot Subt Catalogs: {len(subt_phot_catalogs)}")

		ssf_regions = sorted(glob.glob(f"{path_data}/*com.ssf.reg"))
		print(f"SSF Regions: {len(ssf_regions)}")
		ssf_catalogs = sorted(glob.glob(f"{path_data}/*com.ssf.txt"))
		print(f"SSF Catalogs: {len(ssf_catalogs)}")
		#------------------------------------------------------------
		#	Transient Candidates
		#------------------------------------------------------------
		#	calib_7DT01_T12936_20240423_034610_r_360.com.437.sci.fits
		sci_snapshots = sorted(glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.com.*.sci.fits"))
		#	ref_PS1_T12936_00000000_000000_r_0.conv.437.ref.fits
		ref_snapshots = sorted(glob.glob(f"{path_data}/ref_*_{obj}_*_*_{filte}_*.conv.*.ref.fits"))
		#	calib_7DT01_T12936_20240423_034610_r_360.com.subt.437.sub.fits
		sub_snapshots = sorted(glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.subt.*.sub.fits"))
		print(f"Snapshots (fits): {len(sci_snapshots)}, {len(ref_snapshots)}, {len(sub_snapshots)}")

		sci_png_snapshots = sorted(glob.glob(f"{path_data}/*.sci.png"))
		ref_png_snapshots = sorted(glob.glob(f"{path_data}/*.ref.png"))
		sub_png_snapshots = sorted(glob.glob(f"{path_data}/*.sub.png"))
		print(f"Snapshots (png): {len(sci_png_snapshots)}, {len(ref_png_snapshots)}, {len(sub_png_snapshots)}")

		#	calib_7DT01_T09373_20240423_033207_r_360.com.subt.flag.summary.csv
		flag_tables = sorted(glob.glob(f"{path_data}//calib_*_{obj}_*_*_{filte}_*.subt.flag.summary.csv"))
		print(f"Flag Summary Tables: {flag_tables}")
		#	calib_7DT01_T12931_20240423_031935_r_360.com.subt.transient.cat
		transient_catalogs = sorted(glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.com.subt.transient.cat"))
		print(f"Transient Catalogs: {len(transient_catalogs)}")
		transient_regions = sorted(glob.glob(f"{path_data}/calib_*_{obj}_*_*_{filte}_*.com.subt.transient.reg"))
		print(f"Transient Regions: {len(transient_regions)}")
		#------------------------------------------------------------
		#	Header Files
		#------------------------------------------------------------
		header_files = sorted(glob.glob(f"{path_data}/*.head"))
		print(f"Header Files: {len(header_files)}")
		#	Grouping files
		files_to_base = single_frames + stack_frames
		files_to_phot = single_phot_catalogs + single_phot_pngs \
			+ stack_phot_catalogs + stack_phot_pngs
		files_to_transient = convolved_ref_frames + subt_frames + subt_phot_catalogs \
			+ ssf_regions + ssf_catalogs \
			+ transient_catalogs + transient_regions + flag_tables
		files_to_candidate_fits = sci_snapshots + ref_snapshots + sub_snapshots
		files_to_candidate_png = sci_png_snapshots + ref_png_snapshots + sub_png_snapshots		
		files_to_header = header_files

		destination_paths = [
			path_destination, 
			path_phot,
			path_transient, 
			path_transient_cand_fits, 
			path_transient_cand_png,
			path_header,
			]

		all_files = [
			files_to_base, 
			files_to_phot,
			files_to_transient, 
			files_to_candidate_fits, 
			files_to_candidate_png,
			files_to_header,
			]
		#	Move
		# with ThreadPoolExecutor(max_workers=ncore) as executor:
		# 	for files, destination in zip(all_files, destination_paths):
		# 		executor.map(lambda file_path: move_file(file_path, destination), files)
		# print("Done")

		with ThreadPoolExecutor(max_workers=ncore) as executor:
			for files, destination in zip(all_files, destination_paths):
				# move_file 함수를 partial을 이용해 목적지 경로를 고정합니다.
				move_func = partial(move_file, destination_folder=destination)
				executor.map(move_func, files)



#	Report the LOG

In [ ]:
end_localtime = time.strftime('%Y-%m-%d_%H:%M:%S_(%Z)', time.localtime())
note = "-"
log_text = f"{path_new},{start_localtime},{end_localtime},{note}\n"

with open(path_log, 'a') as file:
	file.write(f"{log_text}")

objtbl.write(f"{path_data}/data_processing.log", format='csv')